<h1 style="text-align:center;">Numerical Quantum Mechanics</h1>
<h2 style="text-align:center;">Time-Independent Schrodinger Equation using Finite Difference</h2>
<h3 style="text-align:center;">Fadjar Fathurrahman</h3>
<h3 style="text-align:center;">Mariya Al Qibtiya Nasution</h3>

In [1]:
import PyPlot
const plt = PyPlot;

In [2]:
plt.svg(true);

In [3]:
using LinearAlgebra

In [8]:
using SparseArrays

Let's remind 

In [4]:
function build_D2_matrix(N::Int64, h::Float64)
    mat = zeros(Float64,N,N)
    for i = 1:N
        mat[i,i] = -2.0
        if i != N
            mat[i,i+1] = 1.0
            mat[i+1,i] = mat[i,i+1]
        end
    end
    return mat/h^2
end

build_D2_matrix (generic function with 1 method)

In [43]:
Nx = 30
x_min = -5.0
x_max = 5.0
Lx = x_max - x_min
hx = Lx/(Nx-1)
x = zeros(Float64,Nx)
for i in 1:Nx
    x[i] = x_min + (i-1)*hx
end

In [44]:
Ny = 30
y_min = -5.0
y_max = 5.0
Ly = y_max - y_min
hy = Ly/(Ny-1)
y = zeros(Float64,Ny)
for j in 1:Ny
    y[j] = y_min + (j-1)*hy
end

In [56]:
Npoints = Nx*Ny
r = zeros(2,Npoints)
ip = 0
idx_ip2xy = zeros(Int64,2,Npoints)
idx_xy2ip = zeros(Int64,Nx,Ny)
for j in 1:Ny
    for i in 1:Nx
        ip = ip + 1
        r[1,ip] = x[i]
        r[2,ip] = y[j]
        idx_ip2xy[1,ip] = i
        idx_ip2xy[2,ip] = j
        idx_xy2ip[i,j] = ip
    end
end

In [47]:
D2x = build_D2_matrix(Nx, hx);
D2y = build_D2_matrix(Ny, hy);

In [53]:
∇2 = kron(D2x, speye(Ny)) + kron(speye(Nx), D2y);

In [54]:
∇2.m, ∇2.n

(900, 900)

In [25]:
ψ1 = rand(Nx*Ny);

In [26]:
∇2*ψ1;

In [57]:
function pot_harmonic(x, y, ω=1.0)
    return 0.5 * ω^2 *( x^2 + y^2 )
end

pot_harmonic (generic function with 6 methods)

In [58]:
Vpot = zeros(Npoints)
for i in 1:Npoints
    Vpot[i] = pot_harmonic(r[1,i], r[2,i])
end

In [55]:
Npoints

441

In [59]:
Ham = -0.5*∇2 + spdiagm( 0 => Vpot );

In [27]:
using IterativeSolvers

In [60]:
res = lobpcg(Ham, false, 5);

In [61]:
res.λ

5-element Array{Float64,1}:
 0.9925118683851903
 1.9774185788069665
 1.9774185788122849
 2.9469286531861423
 2.946928655014062 

In [62]:
res

Results of LOBPCG Algorithm
 * Algorithm: LOBPCG - CholQR
 * λ: [0.9925118683851903,1.9774185788069665, ...]
 * Residual norm(s): [4.767635709620756e-6,1.3836270471368007e-5, ...]
 * Convergence
   * Iterations: 92
   * Converged: true
   * Iterations limit: 200


In [28]:
?lobpcg

search: lobpcg lobpcg! LOBPCGIterator



The Locally Optimal Block Preconditioned Conjugate Gradient Method (LOBPCG)

Finds the `nev` extremal eigenvalues and their corresponding eigenvectors satisfying `AX = λBX`.

`A` and `B` may be generic types but `Base.mul!(C, AorB, X)` must be defined for vectors and strided matrices `X` and `C`. `size(A, i::Int)` and `eltype(A)` must also be defined for `A`.

```
lobpcg(A, [B,] largest, nev; kwargs...) -> results
```

# Arguments

  * `A`: linear operator;
  * `B`: linear operator;
  * `largest`: `true` if largest eigenvalues are desired and false if smallest;
  * `nev`: number of eigenvalues desired.

## Keywords

  * `log::Bool`: default is `false`; if `true`, `results.trace` will store iterations   states; if `false` only `results.trace` will be empty;
  * `P`: preconditioner of residual vectors, must overload `ldiv!`;
  * `C`: constraint to deflate the residual and solution vectors orthogonal   to a subspace; must overload `mul!`;
  * `maxiter`: maximum number of iterations; default is 200;
  * `tol::Real`: tolerance to which residual vector norms must be under.

# Output

  * `results`: a `LOBPCGResults` struct. `r.λ` and `r.X` store the eigenvalues and eigenvectors.

---

```
lobpcg(A, [B,] largest, X0; kwargs...) -> results
```

# Arguments

  * `A`: linear operator;
  * `B`: linear operator;
  * `largest`: `true` if largest eigenvalues are desired and false if smallest;
  * `X0`: Initial guess, will not be modified. The number of columns is the number of eigenvectors desired.

## Keywords

  * `not_zeros`: default is `false`. If `true`, `X0` will be assumed to not have any all-zeros column.
  * `log::Bool`: default is `false`; if `true`, `results.trace` will store iterations   states; if `false` only `results.trace` will be empty;
  * `P`: preconditioner of residual vectors, must overload `ldiv!`;
  * `C`: constraint to deflate the residual and solution vectors orthogonal   to a subspace; must overload `mul!`;
  * `maxiter`: maximum number of iterations; default is 200;
  * `tol::Real`: tolerance to which residual vector norms must be under.

# Output

  * `results`: a `LOBPCGResults` struct. `r.λ` and `r.X` store the eigenvalues and eigenvectors.

---

lobpcg(A, [B,] largest, X0, nev; kwargs...) -> results

# Arguments

  * `A`: linear operator;
  * `B`: linear operator;
  * `largest`: `true` if largest eigenvalues are desired and false if smallest;
  * `X0`: block vectors such that the eigenvalues will be found size(X0, 2) at a time;   the columns are also used to initialize the first batch of Ritz vectors;
  * `nev`: number of eigenvalues desired.

## Keywords

  * `log::Bool`: default is `false`; if `true`, `results.trace` will store iterations   states; if `false` only `results.trace` will be empty;
  * `P`: preconditioner of residual vectors, must overload `ldiv!`;
  * `C`: constraint to deflate the residual and solution vectors orthogonal   to a subspace; must overload `mul!`;
  * `maxiter`: maximum number of iterations; default is 200;
  * `tol::Real`: tolerance to which residual vector norms must be under.

# Output

  * `results`: a `LOBPCGResults` struct. `r.λ` and `r.X` store the eigenvalues and eigenvectors.


In [9]:
speye(N::Int64) = sparse(Matrix(1.0I, N, N))

speye (generic function with 1 method)

In [10]:
speye(3)

3×3 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [1, 1]  =  1.0
  [2, 2]  =  1.0
  [3, 3]  =  1.0

In [9]:
x, y

([-5.0, -2.5, 0.0, 2.5, 5.0], [-5.0, -2.5, 0.0, 2.5, 5.0])

Setup the grid:

In [19]:
Npoints = Nx*Ny
r = zeros(2,Npoints)
ip = 0
idx_ip2xy = zeros(Int64,2,Npoints)
idx_xy2ip = zeros(Int64,Nx,Ny)
for j in 1:Ny
    for i in 1:Nx
        ip = ip + 1
        r[1,ip] = x[i]
        r[2,ip] = y[j]
        idx_ip2xy[1,ip] = i
        idx_ip2xy[2,ip] = j
        idx_xy2ip[i,j] = ip
    end
end

This is bebebeb

$$
\alpha + \beta + \Gamma
$$

In [ ]:
struct FD2dGrid
    Nx::Int64
    Ny::Int64
    hx::Float64
    hy::Float64
    r::Array{Float64,2}
    idx_ip2xy::Array{Int64,2}
    idx_xy2ip::Array{Int64,2}
end